## Notice.

<br>

#### * **추후 SoundCloud 측에서 HTML 코드를 수정할 경우 일부 작동이 불가할 수 있습니다.** 

<br>

In [1]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

from selenium import webdriver
from bs4 import BeautifulSoup

import pandas as pd
import time
import datetime 
from dateutil import rrule

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

<br>
<br>

# 1. Artist

In [ ]:
# 수집할 데이터

# name_display
# name_real # 없다면 생략
# location # 없다면 생략
# num_follower
# num_following
# num_track

In [2]:
artist = 'gravez' # artist 이름만 바꿔주면 다른 아티스트 정보를 크롤링할 수 있습니다.
url = "https://soundcloud.com/" + artist 

In [3]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

driver.get(url) 
time.sleep(3)

web = driver.page_source # URL로 접속한 후 HTML code를 얻어옵니다. 
source = BeautifulSoup(web, 'html.parser')
# print(source) # source를 그대로 출력할 경우, 담고 있는 텍스트가 무척 많아서 버벅이게 될 수 있습니다.



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [C:\Users\pc\.wdm\drivers\chromedriver\win32\98.0.4758.102\chromedriver.exe] found in cache


In [4]:
# Selenium으로 켠 가상 브라우저는 활용을 마칠 때마다 아래 명령어로 닫아주는 것이 메모리 관리에 좋습니다.

driver.close()
driver.quit()

In [5]:
# source.find('HTML 태그명', {'class':'해당 태그의 클래스 값'}) 
# -> 특정 클래스 값을 가진 HTML 태그를 1개 얻어옵니다 (여러개의 태그가 동일한 클래스값을 가지고 있을 시 가장 첫번째 태그)

# source.find_all('HTML 태그명', {'class':'해당 태그의 클래스 값'})
# -> 특정 클래스 값을 가진 HTML 태그를 모두 얻어옵니다 (list가 됩니다.)

name_display = source.find('h3', {'class':'profileHeaderInfo__userName'})

if name_display is not None:
    name_display = name_display.get_text() # HTML 태그에서 텍스트 값만 꺼냅니다.
    if 'Pro' in name_display: 
        name_display = name_display.replace('Pro', '') # 'gravez Pro' -> 'gravez '
    name_display = name_display.strip() # 앞/뒤의 공백을 제거합니다.

print(name_display)

None


In [32]:
# name_real(실제 이름)과 location(city & country)가 동일한 형태의 HTML 태그 안에 담겨 있습니다.
# 각 case별로, 
# 1) 만약 name_real과 location 정보가 다 있을 경우에는 2가지 정보를 별도로 나눠 catch합니다.
# 2) name_real과 location 둘 중 1개의 정보만 있을 때에는 그 정보가 name_real인지 location인지 기술적으로 분간할 수 없습니다.
# 3) 다만 여러 artist들을 살펴본 결과 대부분 1개의 정보만 있을 때에는 그 정보가 location이었기에 location으로 처리하도록 하였습니다.

artist_additional_info = source.find_all('h4', {'class':'profileHeaderInfo__additional'})

if len(artist_additional_info) == 2: # name_real과 location 정보가 다 있을 경우
    name_real = artist_additional_info[0].get_text()
    location = artist_additional_info[1].get_text()

elif len(artist_additional_info) == 1: # name_real과 location 둘 중 1개의 정보만 있을 경우
    name_real = ''
    location = artist_additional_info[0].get_text()
    
elif (len(artist_additional_info) == 0) or (artist_additional_info is None): # name_real과 location가 모두 없을 경우
    name_real = ''
    location = ''
    
print(name_real)
print(location)

gravez
los angeles, ca


In [37]:
artist_nums = source.find_all('div', {'class':'infoStats__value'}) # 팔로워 수, 팔로잉 수, 트랙 수가 모두 모여있습니다.

num_follower = artist_nums[0].get_text()
num_following = artist_nums[1].get_text()
num_track = artist_nums[2].get_text()

print(num_follower)
print(num_following)
print(num_track)

89.8K
51
9


In [38]:
# result

print(name_display)
print(name_real) # 없다면 생략
print(location) # 없다면 생략
print(num_follower)
print(num_following)
print(num_track)

gravez
gravez
los angeles, ca
89.8K
51
9


<br>
<br>

# 2. Track

In [ ]:
# 수집할 데이터

# diff_year # 현재 날짜와의 차이 중 'n년' 단위의 값 (ex. 0년 6개월 전 업로드 -> 0)
# diff_month # 현재 날짜와의 차이 중 'n년'을 제외한 나머지 개월 수 (ex. 3년 3개월 전 업로드 -> 3) <- 웹 페이지 화면에는 노출되지 않으나 가능
# genre # 옵션이라 없을 수도 있다는 내용을 반영했습니다.
# num_play 
# num_like
# num_repost

# track_name : track 이름 (새로 추가했습니다.)
# track_url : track URL (새로 추가했습니다. 추후 comment 수집을 위해 활용됩니다.)

In [39]:
artist = 'gravez' # artist 이름만 바꿔주면 다른 아티스트의 트랙 정보를 크롤링할 수 있습니다.
url = "https://soundcloud.com/" + artist + '/tracks'

In [45]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

driver.get(url) 
time.sleep(3)

# 아래 코드가 특정 웹페이지에서 Selenium을 활용해 스크롤 다운하는 코드입니다.
# 가장 아래까지 스크롤 다운 후 매번 3초씩 로딩을 기다린 후 스크롤 다운을 마지막까지 진행합니다.
while True:
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
         break

web = driver.page_source
source = BeautifulSoup(web, 'html.parser')
# print(source) # source를 그대로 출력할 경우, 담고 있는 텍스트가 무척 많아서 버벅이게 될 수 있습니다.

In [46]:
driver.close()
driver.quit()

In [51]:
tracks = source.find_all('div', {'class':'sound__content'})
print(len(tracks)) # 총 트랙의 수

9


In [106]:
track_name = []
diff_year = []
diff_month = []
genre = [] 
num_play = []
num_like = []
num_repost = []
track_url = []

if tracks is not None: # 트랙이 하나도 없을 경우를 대비 
    for track in tracks:

        # 1) Track 이름
        track_name.append(track.find('a', {'class':'soundTitle__title'}).get_text().strip())

        # 2) 업로드 날짜와 현재 날짜와의 차이 (연 & 월)
        # 특정 HTML 태그에 대해 .get_text()는 내부 텍스트를 꺼내지만, .attrs는 태그의 Attribute들을 꺼낼 수 있습니다. 
        uploaded_date = track.find('time', {'class':'relativeTime'}).attrs['datetime'][:10] # '2020-06-21'
        uploaded_date = datetime.datetime.strptime(uploaded_date, '%Y-%m-%d') # 파이썬의 datetime 형식으로 변환
        today_date  = datetime.datetime.now()
        
        # 파이썬의 datetime 형식을 갖고 있는 2개의 날짜에 대해 아래 rrule.rrule을 활용해 년도 기준 차이 & 개월 기준 차이를 계산할 수 있습니다.
        # 실제 rrule.rrule의 계산 결과 값이 웹사이트 내 표기된 연/월 수보다 1씩 크므로 1씩 차감해줍니다.
        diff_year.append(len(list(rrule.rrule(rrule.YEARLY, dtstart=uploaded_date, until=today_date))) - 1)
        # 아래와 같이 12로 나눈 나머지를 계산하지 않으면 트랙 업로드 일자부터 오늘 일자까지의 총 개월 수를 계산해버립니다.
        diff_month.append((len(list(rrule.rrule(rrule.MONTHLY, dtstart=uploaded_date, until=today_date))) - 1) % 12)

        # 3) 장르 
        genre_tag = track.find('span', {'class':'soundTitle__tagContent'})

        if genre_tag is not None: # 장르가 없을 경우를 대비
            genre.append(genre_tag.get_text().strip())
        else:
            genre.append('')

        # 4) 재생 횟수
        num_play.append(track.find('li', {'class':'sc-ministats-item'}).attrs['title'].replace(' plays', ''))

        # 5) like 수 (K로 표기된 값들은 plain number 형태로 변환하였습니다.)
        likes = track.find('button', {'class':'sc-button-like'}).get_text()
        if ('K' in likes) and ('.' in likes): # 10.6K -> 10,600
            likes = likes.replace('.', ',')
            likes = likes.replace('K', '00') 
        elif ('K' in likes) and ('.' not in likes): # 13K -> 13,000
            likes = likes.replace('K', ',000')
        num_like.append(likes)

        # 6) repost 수
        num_repost.append(track.find('button', {'class':'sc-button-repost'}).get_text())
        
        # 7) track URL 
        track_simple = track.find('a', {'class':'soundTitle__title'}).attrs['href']
        track_url.append('https://soundcloud.com' + track_simple)

In [114]:
# result

# 아래 코드로 각 리스트들을 Pandas DataFrame의 각 열로 넣어 DataFrame을 만들 수 있습니다.
artist_track_df = pd.DataFrame({
    'artist_name' : [artist for i in range(len(track_name))],
    'track_name' : track_name,
    'diff_year' : diff_year,
    'diff_month' : diff_month,
    'genre' : genre,
    'num_play' : num_play,
    'num_like' : num_like,
    'num_repost' : num_repost,
    'track_url' : track_url    
})

artist_track_df

artist_name                              track_name  diff_year  diff_month  \
0      gravez                                   swang          0           3   
1      gravez                        closed on sunday          0           3   
2      gravez                            dreamcatcher          1           9   
3      gravez                                   taste          2           0   
4      gravez      GoldLink - Zipporah (Gravez Remix)          4           8   
5      gravez  Cousin Stizz - Shoutout (Gravez Remix)          5           1   
6      gravez             Where We From *RIP Natasja*          5           6   
7      gravez       Jaden Smith - Fast (Gravez Remix)          5          11   
8      gravez   Rae Sremmurd - No Type (Gravez Remix)          6           0   

             genre   num_play num_like num_repost  \
0       R&B & Soul     62,578    2,675        298   
1          Worship     77,424    3,077        388   
2   merrychristmas    196,364    5,933        845   
3      Doo Doo Doo    293,782   10,600      1,497   
4           AATWDT  1,726,837   42,000      8,590   
5              $$$  1,673,796   34,300      5,562   
6         SangoOde  1,642,427   41,600      8,139   
7  WhyUDriveSoFast  1,406,345   28,300      5,424   
8             Flex  1,473,003   23,200      4,939   

                                           track_url  
0                https://soundcloud.com/gravez/swang  
1     https://soundcloud.com/gravez/closed-on-sunday  
2         https://soundcloud.com/gravez/dreamcatcher  
3            https://soundcloud.com/gravez/lil-taste  
4  https://soundcloud.com/gravez/goldlink-zippora...  
5  https://soundcloud.com/gravez/cousin-stizz-sho...  
6        https://soundcloud.com/gravez/where-we-from  
7  https://soundcloud.com/gravez/jaden-smith-fast...  
8  https://soundcloud.com/gravez/no-type-gravez-r...

In [150]:
# DataFrame은 아래와 같은 코드로 엑셀 파일로 저장 가능합니다.

artist_track_df.to_excel('artist_track_{}.xlsx'.format(artist), index=False, encoding='utf-8')

# 엑셀 파일을 열고자 할 때 활용할 수 있는 코드
# df = pd.read_excel('artist_track_gravez.xlsx', encoding='utf-8')
# df

In [108]:
print(track_url[0]) # Comment level data 로의 연결점입니다.

https://soundcloud.com/gravez/swang


<br>
<br>

# 3. Comment

In [116]:
# 수집할 데이터

# track_name 
# num_comments # 화면에 적혀있는 코멘트 수(61)보다 실제 화면에 출력되는 코멘트 수(50)가 종종 적은 것 같습니다. (삭제된 코멘트 포함 혹은 숨김처리된 코멘트 등)

# comment_text # 코멘트에 대한 리코멘트 역시도 트랙에 대한 코멘트라고 판단되므로 일반 코멘트로서 동일하게 수집하였습니다. (별도로 다른 형태를 갖고 있지 않습니다.)
# comment_stamp
# comment_diff_year
# comment_diff_month

In [115]:
track_url_sample = track_url[0] # https://soundcloud.com/gravez/swang
track_url_sample

# track_url_sample = 'https://soundcloud.com/gravez/where-we-from'
# track_url_sample

'https://soundcloud.com/gravez/swang'

In [128]:
service = Service(executable_path=ChromeDriverManager().install()) 
driver = webdriver.Chrome(service=service)

driver.get(track_url_sample) 
time.sleep(3)

while True:
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    newHeight = driver.execute_script("return document.body.scrollHeight")
    if newHeight == lastHeight:
         break

web = driver.page_source
source = BeautifulSoup(web, 'html.parser')
# print(source) # source를 그대로 출력할 경우, 담고 있는 텍스트가 무척 많아서 버벅이게 될 수 있습니다.

In [129]:
driver.close()
driver.quit()

In [130]:
track_name = source.find('span', {'class':'soundTitle__title'}).get_text().strip()
num_comments = source.find('span', {'class':'commentsList__actualTitle'}).get_text().replace(' comments', '')

print(track_name)
print(num_comments)

swang
61


In [144]:
comments = source.find_all('div', {'class':'commentItem'}) # 각 코멘트들이 div 태그 박스 안에 담겨있습니다.
# print(len(comments))

comment_text = []
comment_stamp = []
comment_diff_year = []
comment_diff_month = []

if len(comments) != 0: # 코멘트가 하나도 없을 경우를 대비 
    for comment in comments:
        # 1) 코멘트 내용
        comment_text.append(comment.find('div', {'class':'commentItem__body'}).get_text().strip())

        # 2) Timestamp (트랙 내 시간)
        comment_stamp.append(comment.find('a', {'class':'commentItem__timestampLink'}).get_text())

        # 3) 코멘트 등록 날짜와 현재 날짜와의 차이 (연 & 월) : 위 Track에서의 코드에 대한 설명을 참고해주세요.
        commented_date = comment.find('time', {'class':'relativeTime'}).attrs['datetime'][:10]
        commented_date = datetime.datetime.strptime(commented_date, '%Y-%m-%d')
        today_date  = datetime.datetime.now()

        comment_diff_year.append(len(list(rrule.rrule(rrule.YEARLY, dtstart=commented_date, until=today_date))) - 1)
        comment_diff_month.append((len(list(rrule.rrule(rrule.MONTHLY, dtstart=commented_date, until=today_date))) - 1) % 12)

In [147]:
# result

print('Track :', track_name)
print('# of comments :', num_comments)
print('# of comments (real) :', len(comments))

track_comment_df = pd.DataFrame({
    'comment_text' : comment_text,
    'comment_stamp' : comment_stamp,
    'comment_diff_year' : comment_diff_year,
    'comment_diff_month' : comment_diff_month
})

track_comment_df

Track : swang
# of comments : 61
# of comments (real) : 50


comment_text comment_stamp  \
0   Gravez doesnt always upload...but when he does...          0:11   
1                                               so so          0:54   
2                  You can thank 2k for me being here          2:28   
3                                       Straight heat          0:49   
4                                        when u vibin          3:06   
5                                                  ??          0:19   
6                                                FIRE          2:32   
7                                               ????/          0:17   
8                               like this vibe you on          3:16   
9                      Get this one on spotify please          0:47   
10                              o r i g i n a l i t y          1:03   
11           I fuck with this so heavy bruh yes 🔥🔥🔥🔥🔥          0:25   
12                   m gonna need this on Apple Music          0:21   
13                                               guys          2:52   
14                                      Inspo forever          2:39   
15                                      Taiviw tw. Oc          1:35   
16                        This kinda tuff not gon cap          2:44   
17                                             Goated          1:37   
18                                             Wavy 👹          0:59   
19  ❗️I KNOW YALL THINKING DAMN ANOTHER TRASH ASS ...          0:45   
20              Smoking weed you know im getting high          0:26   
21                        The best raper in the world          2:50   
22                                                 ok          0:11   
23                                             ^^**+)          0:51   
24                              should i quit rapping          2:31   
25                                            niieccc          1:29   
26                                          butter!!!          1:53   
27                       (づ■.■)づthat the vibe (￢ ᴥ￢ )          1:15   
28     Yes. If you made it this far, you a real one✊🏼          2:30   
29                                       goat is back          0:20   
30                               This is too niicccee          0:05   
31                                            In love          0:00   
32                                            so glad          1:03   
33                                            G O A T          0:00   
34                                      highway vibez          1:50   
35                                          king shit          0:43   
36                               @vincebyvince: facts          0:43   
37                                 Came from Twitter.          0:05   
38                                          ababababa          0:53   
39                                            K I N G          0:32   
40                                        dis cleannn          0:01   
41                                   He’s backkkk!!!!          0:40   
42                              the gawd has returned          1:07   
43                                            Bout it          0:00   
44                              instant mfing classic          0:38   
45                   This sh!t feel illegal! Got DAMN          0:00   
46                                               Lmao          0:00   
47                                             Nasty。          0:02   
48                                     classic gravez          0:24   
49                                              first          0:43   

    comment_diff_year  comment_diff_month  
0                   0                   2  
1                   0                   2  
2                   0                   2  
3                   0                   2  
4                   0                   2  
5                   0                   2  
6                   0                   2  
7                   0                   2  
8                   0  

In [152]:
track_comment_df.to_excel('track_comment_{}_{}.xlsx'.format(artist, track_name), index=False, encoding='utf-8')